In [9]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import json
import string
import IPython.display as ipd

In [14]:
vocabs = list(string.ascii_lowercase + string.digits) + [' ']
len(vocabs)

37

In [3]:
base_directory = '/home/husein/speech-bahasa'

In [6]:
sani = glob(f'{base_directory}/clean/**/*.wav', recursive = True)
len(sani)

329515

In [23]:
sanis = []
for i in tqdm(sani):
    with open(i[:-4] + '.txt') as fopen:
        text = fopen.read()
    if len(text):
        sanis.append((i, text))
    
len(sanis)

100%|██████████| 329515/329515 [00:08<00:00, 38847.13it/s]


329359

In [24]:
audios, texts = zip(*sanis)

In [25]:
len(texts)

329359

In [26]:
import unicodedata
import re
import itertools

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [27]:
processed_text = [preprocessing_text(t) for t in tqdm(texts)]

100%|██████████| 329359/329359 [00:13<00:00, 24620.62it/s]


In [29]:
with open('bahasa-asr-train.json') as fopen:
    train = json.load(fopen)
    
len(train['X'])

1404435

In [31]:
train['X'][:10]

['/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20425643.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_19424182.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_21080203.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20483314.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20483312.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20483313.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20211965.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20211967.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20211969.mp3',
 '/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_19376535.mp3']

In [34]:
X = train['X'] + list(audios)
Y = train['Y'] + processed_text
len(X), len(Y)

(1733794, 1733794)

In [36]:
import os

In [39]:
exists = [os.path.exists(f) for f in tqdm(X)]

100%|██████████| 1733794/1733794 [00:11<00:00, 145618.91it/s]


In [43]:
sum(exists) / len(exists) == 1.0

True

In [46]:
from sklearn.utils import shuffle

X, Y = shuffle(X, Y)

In [47]:
with open('bahasa-asr-train-combined.json', 'w') as fopen:
    json.dump({'X': X, 'Y':Y}, fopen)

In [35]:
# import malaya_speech

In [36]:
# tokenizer = malaya_speech.subword.generate_tokenizer(processed_text, max_subword_length = 3)

In [37]:
# malaya_speech.subword.save(tokenizer, 'transducer.subword')

In [38]:
# tokenizer = malaya_speech.subword.load('transducer.subword')
# malaya_speech.subword.encode(tokenizer, 'i hate', add_blank = True)

In [39]:
# malaya_speech.subword.decode(tokenizer, [0, 2, 133, 875])

In [40]:
# from pydub import AudioSegment
# import numpy as np

# sr = 16000

# def mp3_to_wav(file, sr = sr):
#     audio = AudioSegment.from_file(file)
#     audio = audio.set_frame_rate(sr).set_channels(1)
#     sample = np.array(audio.get_array_of_samples())
#     return malaya_speech.astype.int_to_float(sample), sr

# def generator(maxlen = 18, min_length_text = 2):
#     for i in tqdm(range(len(audios))):
#         try:
#             if audios[i].endswith('.mp3'):
#                 wav_data, _ = mp3_to_wav(audios[i])
#             else:
#                 wav_data, _ = malaya_speech.load(audios[i])
                
#             if (len(wav_data) / sr) > maxlen:
#                 # print(f'skipped audio too long {audios[i]}')
#                 continue
                
#             if len(processed_text[i]) < min_length_text:
#                 print(f'skipped text too short {audios[i]}')
#                 continue    

#             yield {
#                 'waveforms': wav_data.tolist(),
#                 'waveform_lens': [len(wav_data)],
#                 'targets': malaya_speech.subword.encode(tokenizer, processed_text[i], add_blank = False),
#             }
#         except Exception as e:
#             print(e)
            
# generator = generator()

In [41]:
# import os
# import tensorflow as tf

# os.system('rm bahasa-asr/data/*')
# DATA_DIR = os.path.expanduser('bahasa-asr/data')
# tf.gfile.MakeDirs(DATA_DIR)

In [42]:
# shards = [{'split': 'train', 'shards': 1000}]

In [43]:
# import malaya_speech.train as train

# train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')